In [30]:
import pandas as pd
import numpy as np
import multiprocessing as mp
from pykrx import stock

## ETF 종목 가져오기
- 목표 : 최하위 ETF 종목 선택하기

In [31]:
# 날짜 설정 
start_date = '20220101'
end_date = '20241231'

# 모든 ETF 티커 리스트 가져오기
etf_list = stock.get_etf_ticker_list()
etf_name = [stock.get_etf_ticker_name(etf) for etf in etf_list]

In [32]:
import multiprocessing

print(multiprocessing.cpu_count())

12


In [33]:
# ETF 종가 데이터 가져오기
def fetch_etf_data(etf_number):
    try:
        data = stock.get_etf_ohlcv_by_data(start_date, end_date, etf_number)['종가']
    except Exception as e:
        print(f"Error fetching data for ETF {etf_number}: {e}") # 오류 로그 출력
        return etf_number, None # None을 명시적으로 반환
    
# 멀티프로세싱을 사용하여 병렬로 데이터 가져오기
with mp.Pool(processes=12) as pool:
    results = pool.map(fetch_etf_data, etf_list)

# None 값이 포함된 경우 제외
filtered_results = [(etf,data) for etf, data in results if data is not None]

# 데이터프레임 변환
df_etf = pd.DataFrame({etf: data for etf, data in filtered_results})

print(df_etf.head()) # 결과 확인


Error fetching data for ETF 483230: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 495710: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 442570: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 379780: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 446700: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 417450: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 498860: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 395290: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 473640: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'Error fetching data for ETF 252000: module 'pykrx.stock' has no attribute 'get_etf_ohlcv_by_data'
Error fetching data for ETF 4

In [34]:
df_etf

""


In [35]:
# 각 ETF 종목의 평균 종가 계산
etf_avg_prices = df_etf.mean()

# 평균 종가가 가장 낮은 5 개의 종목 찾기
lowest_5_etfs = etf_avg_prices.nsmallest(5) 

# 결과 출력
print("최하위 5개 ETF 종목:")
print(lowest_5_etfs)

최하위 5개 ETF 종목:
Series([], dtype: float64)


In [36]:
import unittest

In [37]:
from pykrx import stock

# 검색할 ETF 종목 리스트
etf_list = ["204450", "228790"]

# 각 ETF의 구성 종목을 저장할 딕셔너리
etf_portfolio = {}

for etf in etf_list:
    try:
        df = stock.get_etf_portfolio_deposit_file(etf)  # ETF 구성 종목 데이터 가져오기
        
        if df.empty:
            print(f"⚠️ ETF {etf} 데이터가 없습니다.")
            continue
        
        # 컬럼이 무엇이 있는지 출력 (디버깅용)
        print(f"\n📌 ETF {etf}의 데이터 컬럼: {df.columns.tolist()}")

        # "비중(%)" 컬럼이 없을 경우에도 전체 데이터 출력하여 구조 확인
        print(df.head())  # 데이터의 첫 몇 줄 출력

        # "비중(%)" 컬럼이 있다면 정리 후 저장
        if "비중(%)" in df.columns:
            df = df[df["비중(%)"] > 0]  # 비중이 0인 데이터 제거
            df["종목명"] = df.index.map(lambda x: stock.get_market_ticker_name(x) if x.isdigit() else x)
            df = df[["종목명", "비중(%)", "보유수량"]]
            etf_portfolio[etf] = df

    except Exception as e:
        print(f"❌ Error fetching portfolio for ETF {etf}: {e}")

# 결과 출력
for etf, df in etf_portfolio.items():
    print(f"\n📌 ETF {etf} 구성 종목:")
    print(df)



📌 ETF 204450의 데이터 컬럼: ['계약수', '금액', '비중']
            계약수  금액   비중
티커                      
100000  2777.31   0  0.0
017191   735.29   0  0.0
040111   134.45   0  0.0
070341    67.23   0  0.0
100000  7953.78   0  0.0

📌 ETF 228790의 데이터 컬럼: ['계약수', '금액', '비중']
          계약수       금액     비중
티커                           
214450   33.0  9108000  11.27
192820   54.0  8602200  10.32
051900   26.0  8710000  10.17
161890  142.0  8051400   9.90
090430   70.0  8680000   9.89
